# HoloScript Python Bindings Tutorial

Welcome to HoloScript! This notebook demonstrates how to parse, validate, and generate VR scenes using Python.

## Installation

```bash
pip install holoscript
```

In [ ]:
# Install holoscript if not already installed
# !pip install holoscript

from holoscript import (
    parse, 
    validate, 
    list_traits, 
    explain_trait, 
    suggest_traits,
    generate_object,
    generate_scene
)

print("✅ HoloScript loaded successfully!")

## 1. Parsing HoloScript Code

HoloScript supports three file formats:
- `.hs` - Classic object-centric syntax
- `.hsplus` - Extended with VR traits (@grabbable, @networked, etc.)
- `.holo` - Declarative composition format

In [ ]:
# Parse a .holo composition
holo_code = '''
composition "Crystal Garden" {
  environment {
    skybox: "nebula"
    ambient_light: 0.4
  }

  template "Crystal" {
    @grabbable
    @glowing
    geometry: "sphere"
    color: "#00ffff"
    
    state {
      glow_intensity: 1.0
    }
  }

  object "MainCrystal" using "Crystal" {
    position: [0, 1.5, 0]
    scale: [0.5, 0.5, 0.5]
  }

  object "FloatingCrystal" @hoverable {
    geometry: "cone"
    position: [2, 2, 0]
    color: "#ff00ff"
  }
}
'''

result = parse(holo_code)
print(f"Parse success: {result.success}")
print(f"Format detected: {result.format}")
print(f"Objects found: {result.objects}")
print(f"Traits used: {result.traits}")

In [ ]:
# Inspect the AST
import json

print(json.dumps(result.ast, indent=2))

## 2. Parsing .hsplus Code

The `.hsplus` format adds VR-specific traits and physics properties.

In [ ]:
# Parse .hsplus code with traits
hsplus_code = '''
orb MagicOrb @grabbable @throwable @glowing {
    color: "cyan"
    scale: 0.3
    physics: {
        mass: 0.5
        restitution: 0.8
    }
    
    onGrab: {
        haptic.feedback("medium")
        this.glow_intensity = 2.0
    }
    
    onThrow: {
        audio.play("swoosh.wav")
    }
}

cube Platform @collidable {
    position: [0, 0, 0]
    scale: [5, 0.2, 5]
    color: "#404040"
}
'''

result = parse(hsplus_code, format="hsplus")
print(f"Objects: {result.objects}")
print(f"Traits: {result.traits}")

if result.warnings:
    print("\nWarnings:")
    for w in result.warnings:
        print(f"  ⚠️ {w.message}")

## 3. Validating HoloScript

The validator checks syntax, unknown traits, and provides AI-friendly error messages.

In [ ]:
# Valid code
valid_code = '''
composition "Test Scene" {
  object "Cube" @grabbable {
    geometry: "cube"
    color: "#ff0000"
  }
}
'''

result = validate(valid_code)
print(f"Valid: {result.valid}")
print(f"Errors: {len(result.errors)}")
print(f"Warnings: {len(result.warnings)}")

In [ ]:
# Invalid code with errors
invalid_code = '''
composition "Broken" {
  object "Cube" @unknownTrait {
    geometry: "sper"  # typo!
  }
  # Missing closing brace
'''

result = validate(invalid_code)
print(f"Valid: {result.valid}")

for error in result.errors:
    print(f"❌ Line {error.line}: {error.message}")
    if error.suggestion:
        print(f"   💡 Suggestion: {error.suggestion}")

for warning in result.warnings:
    print(f"⚠️ {warning.message}")

## 4. Exploring VR Traits

HoloScript includes 49 built-in VR traits across categories.

In [ ]:
# List all trait categories
all_traits = list_traits()

for category, traits in all_traits.items():
    print(f"\n{category.upper()} ({len(traits)} traits):")
    print(f"  {', '.join(traits[:5])}{'...' if len(traits) > 5 else ''}")

In [ ]:
# Get detailed info about a specific trait
info = explain_trait("grabbable")

print(f"Trait: {info['name']}")
print(f"Category: {info['category']}")
print(f"Description: {info['description']}")
print(f"\nExample:")
print(info['example'])

In [ ]:
# Get AI-powered trait suggestions
suggestions = suggest_traits("I want to pick up a ball and throw it")

print("Suggested traits:")
for s in suggestions:
    print(f"  {s['trait']} ({s['confidence']:.0%}) - {s['reason']}")

## 5. Generating HoloScript

Generate HoloScript code from natural language descriptions.

In [ ]:
# Generate a single object
obj = generate_object(
    name="MagicSword",
    description="A glowing sword that can be grabbed and swung",
    geometry="model/sword.glb"
)

print(obj.code)
print(f"\nSuggested traits: {obj.traits}")

In [ ]:
# Generate a complete scene
scene = generate_scene(
    name="Floating Islands",
    description="A magical archipelago of floating islands with glowing crystals and waterfalls",
    object_count=5
)

print(scene.code)

## 6. Full Workflow Example

Parse → Validate → Export workflow for AI agents.

In [ ]:
def process_holoscript(code: str) -> dict:
    """Complete HoloScript processing pipeline."""
    
    # Step 1: Parse
    parse_result = parse(code)
    if not parse_result.success:
        return {
            "success": False,
            "stage": "parse",
            "errors": [e.message for e in parse_result.errors]
        }
    
    # Step 2: Validate
    validation = validate(code)
    if not validation.valid:
        return {
            "success": False,
            "stage": "validate",
            "errors": [e.message for e in validation.errors],
            "suggestions": [e.suggestion for e in validation.errors if e.suggestion]
        }
    
    # Step 3: Return success with metadata
    return {
        "success": True,
        "format": parse_result.format,
        "objects": parse_result.objects,
        "traits": parse_result.traits,
        "ast": parse_result.ast,
        "warnings": [w.message for w in validation.warnings]
    }

# Test the pipeline
test_code = '''
composition "VR Lounge" {
  environment {
    skybox: "sunset"
    ambient_light: 0.5
  }

  object "Sofa" @collidable {
    geometry: "model/sofa.glb"
    position: [0, 0, -2]
  }

  object "CoffeeTable" @collidable {
    geometry: "cube"
    position: [0, 0.3, -1]
    scale: [0.8, 0.05, 0.5]
  }

  object "Mug" @grabbable @throwable {
    geometry: "model/mug.glb"
    position: [0, 0.35, -1]
    physics: { mass: 0.2 }
  }
}
'''

result = process_holoscript(test_code)
print(f"Success: {result['success']}")
print(f"Format: {result.get('format', 'N/A')}")
print(f"Objects: {result.get('objects', [])}")
print(f"Traits: {result.get('traits', [])}")

## Next Steps

- **Documentation**: [holoscript.dev/python](https://holoscript.dev/python)
- **GitHub**: [github.com/brianonbased-dev/HoloScript](https://github.com/brianonbased-dev/HoloScript)
- **Playground**: [holoscript.dev/playground](https://holoscript.dev/playground)

### For AI Agents

The HoloScript Python bindings are designed for AI integration:

```python
# Grok/Claude/ChatGPT integration pattern
from holoscript import parse, validate, generate_scene

def handle_user_request(prompt: str) -> str:
    scene = generate_scene(name="User Scene", description=prompt)
    result = validate(scene.code)
    
    if result.valid:
        return f"Here's your VR scene:\n\n```holo\n{scene.code}\n```"
    else:
        return f"I generated code but found issues: {result.errors}"
```